In [1]:
import numpy as np
import pandas as pd
from tileserve import compute_prototile_dir, compute_prototile_path, get_layerdef, find_or_generate_layer

2023-06-09 14:03:15-0400 55998 3.425GB: Starting, path /workspace2/rsargent/uwsgi/dotmaptiles-staging-server/tileserve.py
2023-06-09 14:03:15-0400 55998 3.425GB: Containing dir dotmaptiles-staging-server, is_production=False, dotmaptiles_table=dotmaptiles_staging


Connecting to database earthtime with host=/var/run/postgresql
Python version:  3.11.4 (main, Jun  7 2023, 12:45:48) [GCC 11.3.0]
gcc -pthread -shared -rdynamic -fno-strict-aliasing -g -DNDEBUG -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/usr/include/python3.11 -L/home/rsargent/uwsgi/dotmaptiles-staging-server/.venv/lib/python3.11/site-packages -L/home/rsargent/uwsgi/dotmaptiles-staging-server/.venv/lib/python3.11/site-packages/../.. -lpython3.11 /tmp/tmp13o5ydoi.c -o /tmp/tmpf2ml8d1z-000001.so


In [2]:
# X: 0-256 in web mercator
# Y: 0-256 in web mercator
# 1 <= blockidx <= N inclusive, where N is the number of blocks in the county
# 0 <= dotidx < numdots_in_block

record_dtype = np.dtype([('x', np.float32), ('y', np.float32), ('block_idx', np.int32), ('dot_idx', np.int32)])
record_bytes = record_dtype.itemsize
assert record_bytes == 16

In [3]:


dfs = []

geog_year = 2020
levelSubsample = {}

prototile_dir = compute_prototile_dir(geog_year, levelSubsample)
tile_path = compute_prototile_path(prototile_dir, pz=10, px=265, py=414)

df = pd.DataFrame(np.fromfile(tile_path, dtype=record_dtype))
#     "prototiles.2020/10/265/413.bin",
#     "prototiles.2020/10/266/413.bin",
#     "prototiles.2020/10/265/414.bin",
#     "prototiles.2020/10/266/414.bin"


df = df[df['block_idx'] <= 20]
df


2023-06-09 14:03:16-0400 55998 3.427GB: draw_options.levelSubsample does not exist, using prototiles
2023-06-09 14:03:16-0400 55998 3.427GB: using prototiles for geography 2020


,x,y,block_idx,dot_idx
65,66.497932,103.567261,14,12
80,66.492172,103.567741,13,107
94,66.488724,103.567032,10,24
97,66.488342,103.566811,5,21
127,66.496513,103.566299,13,177
...,...,...,...,...
74385,66.490265,103.567413,8,32
74404,66.495361,103.565231,3,3
74483,66.495544,103.566772,13,106
74614,66.493332,103.567871,15,235


In [4]:
layerdef = get_layerdef("census2020_p001_pop_2020_test")
layer = find_or_generate_layer(layerdef, geog_year)
layer['populations']


2023-06-09 14:03:26-0400 55998 3.429GB: census2020_p001_pop_2020_test
2023-06-09 14:03:26-0400 55998 3.429GB: acce6a43f41c10ff3101a6031cceb9d8: computing from #ebc634;census2020_block2020.P0010001 with 2020 geography


TypeError: eval_() missing 1 required positional argument: 'geography_year'

In [ ]:

def LonLatToWebMercator(lon, lat):
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def WebMercatorToLonLat(x,y):
    lat = math.atan(math.exp((128.0 - y) * math.pi / 128.0)) * 360.0 / math.pi - 90.0
    lon = x * 360.0 / 256.0 - 180.0
    return [lon, lat]

lonlats = df.apply(lambda row: WebMercatorToLonLat(row['x'], row['y']), axis=1, result_type='expand')

df = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=gpd.points_from_xy(lonlats[0], lonlats[1]))
# Select only the dots within population (e.g. 10 dots for a block of 10 population)
df = df[df['dot_idx'] < pop_2020[df['block_idx']]]
df.explore()
